# MediaPipe

### 1. 실시간 손 인식 및 추적

In [1]:
import cv2
import mediapipe as mp

# 손 추적을 위한 객체 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1747100414.729650  434659 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro


In [2]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    frame = cv2.flip(frame, 1)    # 좌우반전
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks, hand_lr in zip(results.multi_hand_landmarks, results.multi_handedness):

            label = hand_lr.classification[0].label

            img_h, img_w, _ = frame.shape

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if label == 'Left':
                cv2.putText(frame, 'Left', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif label == 'Right':
                cv2.putText(frame, 'Right', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('hand', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1747100414.739020  434791 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747100414.745666  434791 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747100416.553613  434792 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


### 2. 손가락 사이 거리 측정

In [6]:
import numpy as np

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    frame = cv2.flip(frame, 1)    # 좌우반전
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks, hand_lr in zip(results.multi_hand_landmarks, results.multi_handedness):

            # 손목 좌표
            wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
            print("wrist:", wrist)

            label = hand_lr.classification[0].label

            img_h, img_w, _ = frame.shape

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if label == 'Left':
                cv2.putText(frame, 'Left', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif label == 'Right':
                cv2.putText(frame, 'Right', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            if label == 'Right':
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                thumb_x = thumb_tip.x * img_w
                thumb_y = thumb_tip.y * img_h
                idx_finger_x = index_finger_tip.x * img_w
                idx_finger_y = index_finger_tip.y * img_h

                point1 = np.array([thumb_x, thumb_y])
                point2 = np.array([idx_finger_x, idx_finger_y])

                distance = np.linalg.norm(point2 - point1)

                cv2.putText(frame, str(int(distance)), (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    cv2.imshow('hand', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

wrist: x: 0.700238883
y: 1.09302866
z: 4.34369554e-07

wrist: x: 0.192650601
y: 1.05461955
z: 5.63286164e-07

wrist: x: 0.693749607
y: 1.05884218
z: 4.5329773e-07

wrist: x: 0.191240072
y: 1.0240829
z: 6.50531035e-07

wrist: x: 0.687216
y: 1.01143718
z: 3.90737739e-07

wrist: x: 0.195385173
y: 0.969793499
z: 5.19344894e-07

wrist: x: 0.694566369
y: 0.972198904
z: 3.69021876e-07

wrist: x: 0.20142822
y: 0.923481762
z: 4.24313214e-07

wrist: x: 0.694299042
y: 0.943051
z: 3.21422903e-07

wrist: x: 0.207462072
y: 0.891253293
z: 3.83360714e-07

wrist: x: 0.694226444
y: 0.91773212
z: 3.20897641e-07

wrist: x: 0.211348027
y: 0.873435199
z: 3.67964873e-07

wrist: x: 0.691895664
y: 0.894759238
z: 3.22009782e-07

wrist: x: 0.213778973
y: 0.852940619
z: 3.73999e-07

wrist: x: 0.689926684
y: 0.885233462
z: 2.95894665e-07

wrist: x: 0.21236439
y: 0.842220664
z: 3.50465598e-07

wrist: x: 0.686120272
y: 0.876994491
z: 2.99292793e-07

wrist: x: 0.211572
y: 0.834659934
z: 3.38841801e-07

wrist: x: 0.68

: 

### 3. 손가락 관절 각도 계산

In [4]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    consangle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    arccos = np.arccos(consangle)
    return np.degrees(arccos)

In [5]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    frame = cv2.flip(frame, 1)    # 좌우반전
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks, hand_lr in zip(results.multi_hand_landmarks, results.multi_handedness):

            if1 = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            if2 = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
            if3 = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP]

            if1_list = [if1.x, if1.y, if1.z]
            if2_list = [if2.x, if2.y, if2.z]
            if3_list = [if3.x, if3.y, if3.z]
            index_finger_degree = calculate_angle(if1_list, if2_list, if3_list)

            if not np.isnan(index_finger_degree):
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                cv2.putText(frame, f'Index Finger Angle: {int(index_finger_degree)}',
                            (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)

    cv2.imshow('hand', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()